# Trees
#### Basic binary tree
Implement a nodal tree structure and methods to identify:
- if a node is a leaf
- the values of a node's children
- the values of its grandchildren
- the size of it's subree (the node and all of its descendants)
- the height of its subtree.

In [2]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def is_leaf(self):
        return not self.left and not self.right

    def children(self):
        c = []
        if self.left:
            c.append(self.left.val)
        if self.right:
            c.append(self.right.val)
        return c

    def grandchildren(self):
        g = []
        b = ['left', 'right']
        for child in [self.left, self.right]:
            if child and child.left:
                g.append(child.left.val)
            if child and child.right:
                g.append(child.right.val)
        return g

    def subtree_size(self):
        def component_size(node):
            if node:
                return 1 + component_size(node.left) + component_size(node.right)
            else:
                return 0
        return component_size(self)
    
    def subtree_height(self):
        levels = set()
        def child_check(current_node, l):
            if current_node:
                levels.add(l)
                child_check(current_node.left, l+1)
                child_check(current_node.right, l+1)
            return
        child_check(self,1)
        return max(levels)

In [3]:
my_node = Node(5)
print(f"First node value: {my_node.val}")
print(f"Is first node a leaf? {my_node.is_leaf()}")
print("Adding more nodes to tree...")
my_node.left = Node('l')
my_node.left.right = Node('R')
my_node.right = Node('r')
my_node.right.right = Node('R')
print(f"Is first node still a leaf? {my_node.is_leaf()}")
print(f"What are the first node's children's values? {my_node.children()}")
print(f"And the grandchildren's values? {my_node.grandchildren()}")
print(f"The size of the subtree is now {my_node.subtree_size()} and the height is {my_node.subtree_height()}")
print(f"And the size of the right child's subtree is {my_node.right.subtree_size()} and the height is {my_node.right.subtree_height()}")


First node value: 5
Is first node a leaf? True
Adding more nodes to tree...
Is first node still a leaf? False
What are the first node's children's values? ['l', 'r']
And the grandchildren's values? ['R', 'R']
The size of the subtree is now 5 and the height is 3
And the size of the right child's subtree is 2 and the height is 2
